In [ ]:
# https://ericpony.github.io/z3py-tutorial/guide-examples.htm
!pip install z3-solver

## Introduction

Constraint programming specifies problems by stating conditions or properties -- which are denoted as **constraints** -- that variables must follow. These constraints are partial information the user has about the domain which is used to reduced solution space. A problem stated this way is called a **constraint satisfaction problem** (CSP).

Given a CSP we tipically wish to answer one of the following questions:

+ if it has a solution

+ what is the list of all solutions

+ what is the optimal solution (given a certain criteria)

Problems can be represented in different CSPs. Usually these representations are called **models**. And even if they might be equivalent (in the sense of having the same set of solutions for each problem instance), they might behave differently regarding the computation size.

A CSP consists of:

+ **Variables** where each is identified by an unique name and have a finite **domain**

+ **Constraints** which are predicates regarding one or more variables, that must be satisfied, in order for the problem to have a solution

This notebook uses Python's module `z3` that interacts with Microsoft's Z3, a theorem prover that can be used, among other tasks, for constraint programming.


### Z3 Python Types and API

In [ ]:
from z3 import *

Z3 allows the definition of integer, real or boolean variables that can be used to specify problems.

In [ ]:
x = Int('x')
y = Int('y')

Function `solve` is a shortcut for solving simple systems of constraints,

In [ ]:
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


In [74]:
x, y = Reals('x y')
solve([x >= 0, Or(x + y <= 2.1, x + 2*y >= 6), 
               Or(x + y >= 2  , x + 2*y > 4)])

[y = 21/10, x = 0]


Using reals, Z3 appends a `?` to indicate the value is truncated,

In [75]:
x, y = Reals('x y')
solve(x**2 + y**2 == 3)
set_option(precision=30)
solve(x**2 + y**2 == 3)
set_option(precision=10)

[y = -1.7275343701?, x = 1/8]
[y = -1.727534370135656664238060070969?, x = 1/8]


Function `simplify` asks Z3 to simplify expressions

In [ ]:
print( simplify(x + y + 2*x + 3) )
print( simplify(x < y + x + 2) )
print( simplify((x + y)**3, som=True) )
print( simplify(And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5)) )
print( simplify(And(x == 0, y == 1), elim_and=True) )

3 + 3*x + y
Not(y <= -2)
x*x*x + 3*x*x*y + 3*x*y*y + y*y*y
And(x >= 2, 2*x**2 + y**2 >= 3)
Not(Or(Not(x == 0), Not(y == 1)))


Z3 provides functions for traversing expressions

In [ ]:
n = x + y >= 3 - x

print( "num args: ", n.num_args() )
print( "children: ", n.children() )
print( "1st child:", n.arg(0) )
print( "2nd child's children:", n.arg(1).children() )
print( "operator: ", n.decl() )
print( "op name:  ", n.decl().name() )

num args:  2
children:  [x + y, 3 - x]
1st child: x + y
2nd child's children: [3, x]
operator:  >=
op name:   >=


It's important to distinguish Python expressions like `1/3` with rational Z3 numbers `Q(1,3)`

In [ ]:
x = Real('x')
print( x + 1/3 )
print( x + Q(1,3) )

x + 3333333333333333/10000000000000000
x + 1/3


Boolean variables include the typical operators: And, Or, Not, Implies, and If (the if-then-else),

In [86]:
p, q, r = Bools('p q r')

solve(Implies(p, q), r == Not(q), Or(Not(p), r))

[q = True, p = False, r = False]


In [87]:
simplify(And(p, True))

p

It's possible to fix a set of values (make assumptions) and check what consequences they imply,

In [95]:
a, b, c, d = Bools('a b c d')

s = Solver()
s.add(Implies(a, b), Implies(c, d))   # background formula
print(s.consequences([a, c],          # assumptions
                     [b, c, d]))      # what is implied?

(sat, [Implies(c, c), Implies(a, b), Implies(c, d)])


Z3 also includes a bit vector type

In [ ]:
x = BitVec('x', 16)    # bit vector with 16 bytes
y = BitVec('y', 16)
a = BitVecVal( 20, 16) # constant 20 in binary

solve(x>>2 | y == y & a, x > 10, y > 0)  # &,|,~ are and,or,not

20
[y = 4, x = 16]


Z3 allows for problems with mixed variable types:

In [88]:
p, q = Bools('p q')
x = Real('x')
solve(Or(x < 0, x > 2), Or(p, x**2 == 2), And(q,Not(p)))

[x = -1.4142135623?, p = False, q = True]


### Solvers



Instead of using the `solve` function, it is possible to explicitly instanciate solvers that have a richer state.

In [90]:
x, y = Ints('x y')

s = Solver()
s.add(x>10, y==x+2)

print(s.check()) # sat means the problem is satisfiable

s.push()         # create a new environment, inheriting the previous constraints
s.add(y<11)
print(s.check()) # unsat means the problem is unsatisfiable

s.pop()          # restores to previous state
print(s.check()) 

sat
unsat
sat


If a problem is satisfiable, there is a model with given values for the variables, that makes this CSP true,

In [67]:
x, y, z = Reals('x y z')
s = Solver()
s.add(x > 1, y > 1, x + y > 3, z - x < 10)

if s.check() == sat:
  m = s.model()
  # m[x] is the interpretation/value of variable x in model m
  print('Variable x is %s' % m[x])  
  # get all variables and their interpretations
  for d in m.decls():
    print("%s = %s" % (d.name(), m[d]))

Variable x is 3/2
z = 0
y = 2
x = 3/2


An example of an unsatisfiable model,

In [ ]:
solve(x < 0, x > 0)

no solution


Z3 can solve nonlinear polynomial constraints, but can't solve, say, exponential ones. In these cases, it outputs `unknown`.

In [5]:
x = Real('x')
s = Solver()
s.add(2**x == 3)
s.check()

unknown

### Finite Domains

Consider a problem with two variables $a\in \{1,2,3\}$ and $b\in \{4,6\}$ where no constraints are added. The solution space is the cartesian product of all values:

In [91]:
s = Solver()

a, b = Ints('a b')
s.add(a>=1, a<=3, Or(b==4,b==6))

# prevent next model from using the assignments of previous models
while s.check() == sat:
  print( s.model())
  s.add(Or(a != s.model()[a], b != s.model()[b]))

[b = 4, a = 1]
[b = 4, a = 2]
[b = 4, a = 3]
[b = 6, a = 2]
[b = 6, a = 1]
[b = 6, a = 3]


This can be coded into a function

In [85]:
#ref: https://github.com/0vercl0k/z3-playground/blob/master/z3tools.py
def get_models(s, n_max = None):
  n = 0
  while s.check() == sat:
    if n_max != None and n >= n_max:
      break
    m = s.model()
    yield m

    # Create a new set of constraints blocking the current model
    s.add(Or([sym() != m[sym] for sym in m.decls()]))
    n += 1

s = Solver()
a, b = Ints('a b')
s.add(a>=1, a<=3, Or(b==4,b==6))
for m in get_models(s):
  print(m)    

[b = 4, a = 1]
[b = 4, a = 2]
[b = 4, a = 3]
[b = 6, a = 2]
[b = 6, a = 1]
[b = 6, a = 3]


### Functions

Functions in Z3 have no side-effects and are total.

>  Given a constraints such as x + y > 3, we have been saying that x and y are variables. In many textbooks, x and y are called uninterpreted constants. That is, they allow any interpretation that is consistent with the constraint x + y > 3.

> More precisely, function and constant symbols in pure first-order logic are uninterpreted or free, which means that no a priori interpretation is attached. This is in contrast to functions belonging to the signature of theories, such as arithmetic where the function + has a fixed standard interpretation (it adds two numbers). Uninterpreted functions and constants are maximally flexible; they allow any interpretation that is consistent with the constraints over the function or constant. [ref](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

In [92]:
x, y = Ints('x y')

# define function : Int -> Int
f = Function('f', IntSort(), IntSort())

solve(f(f(x)) == x, f(x) == y, x != y)

[x = 0, y = 1, f = [1 -> 0, else -> 1]]


It's possible to evaluate models,

In [ ]:
x, y = Ints('x y')
f = Function('f', IntSort(), IntSort())

s = Solver()
s.add(f(f(x)) == x, f(x) == y, x != y)
s.check()

m = s.model()
print( "f(x)    =", m.evaluate(f(x)) )
print( "f(f(x)) =", m.evaluate(f(f(x))) )

f(x)    = 1
f(f(x)) = 0


Another eg using quantifiers

In [93]:
x, y = Ints('x y')
a, b = Ints('a b')
f = Function('f', IntSort(), IntSort(), IntSort())

solve(ForAll(x, f(x, x) == 0), f(a, b) == 1)
solve(Exists(x, f(x, y) == 0), f(a, b) == 1)

[b = 2, a = 0, f = [(0, 2) -> 1, else -> 0]]
[b = 5, y = 3, a = 4, f = [(4, 5) -> 1, else -> 0]]


### Satisfiability and Validity

Given a constraint F, F is **valid** if it evaluates to true for any assignment within the domain of its symbols. A valid constraint implies that its negation is **unsatisfiable**.

The next constraint expresses deMorgan rule, making it a valid constraint. If we ask Z3 to satisfy its negation,


In [ ]:
p, q = Bools('p q')
demorgan = And(p, q) == Not(Or(Not(p), Not(q)))

s = Solver()
s.add(Not(demorgan))
s.check()

unsat

We can code this into a function,

In [ ]:
def isProvable(c):
  s = Solver()
  s.add(Not(c))
  return s.check() == unsat

isProvable(demorgan)  

True

## Examples

### Letter Arithmetics

Let's consider the puzzle of finding digits for the following expression,

      SEND
    + MORE
    ------
     MONEY
    
where every letter corresponds to a different digit.

In [68]:
s, m, e, n, d, o, r, y = Ints('s m e n d o r y')
vars = [s,m,e,n,d,o,r,y]
nVars = len(vars)

problem = Solver()

for var in vars[:2]:
  problem.add(var > 0, var < 10)  # S,M cannot be zero

for var in vars[2:]:
  problem.add(var >= 0, var < 10)

problem.add( 1000*(s+m)+100*(e+o)+10*(n+r)+(d+e) == 10000*m+1000*o+100*n+10*e+y )

# all chars represent different digits
problem.add( Distinct(vars) )
#for i in range(nVars):
#  for j in range(i+1, nVars):
#    problem.add( vars[i] != vars[j] )

problem.check()
m = problem.model()
print(m)

[r = 8, s = 9, e = 5, d = 7, n = 6, o = 0, m = 1, y = 2]


### Sudoku Solver

The next code solves a Sudoku puzzle:

In [70]:
# sudoku instance, we use '0' for empty cells
instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

Here the CSP specification:

In [71]:
# ref: https://ericpony.github.io/z3py-tutorial/guide-examples.htm

# 9x9 matrix of integer variables
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ]
      for i in range(9) ]

# each cell contains a value in {1, ..., 9}
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9)
             for i in range(9) for j in range(9) ]

# each row contains a digit at most once
rows_c   = [ Distinct(X[i]) for i in range(9) ]

# each column contains a digit at most once
cols_c   = [ Distinct([ X[i][j] for i in range(9) ])
             for j in range(9) ]

# each 3x3 square contains a digit at most once
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j]
                        for i in range(3) for j in range(3) ])
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c

# force that the solution must have all the non-zero elements of instance
instance_c = [ If(instance[i][j] == 0,
                  True,
                  X[i][j] == instance[i][j])
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(9) ]
    print_matrix(r)
else:
    print("failed to solve")

[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]


### The 8 Queens

The n-Queen problem is a classic CSP: place $n$ Queens on a $n\times n$ board, where there's no mutual attacks.

A model for this problem is to represent a list of columns where columns[i] is the row to place the i-th Queen.

In [ ]:
# ref: https://ericpony.github.io/z3py-tutorial/guide-examples.htm

# We know each queen must be in a different row.
# So, we represent each queen by a single integer: the column position
Q = [ Int('Q_%i' % (i + 1)) for i in range(8) ]

# Each queen is in a column {1, ... 8 }
val_c = [ And(1 <= Q[i], Q[i] <= 8) for i in range(8) ]

# At most one queen per column
col_c = [ Distinct(Q) ]

# Diagonal constraint
diag_c = [ If(i == j,
              True,
              And(Q[i] - Q[j] != i - j, Q[i] - Q[j] != j - i))
           for i in range(8) for j in range(i) ]

solve(val_c + col_c + diag_c)

[Q_5 = 1,
 Q_8 = 7,
 Q_3 = 8,
 Q_2 = 2,
 Q_6 = 3,
 Q_4 = 6,
 Q_7 = 5,
 Q_1 = 4]



### Golomb Ruler

The next model computes the minimal Golomb ruler:

> Golomb ruler is a set of marks at integer positions along an imaginary ruler such that no two pairs of marks are the same distance apart. The number of marks on the ruler is its order, and the largest distance between two of its marks is its length.

To perform this, given a sequence of size $m$, we add variables $d_{i,j}$ with constraints,

+ $d_{i,j} = n_j - n_i$ where $n_i$ is the i-th element of the number sequence.

+ all $d_{i,j}$ must be different

For $m=5$ the results are $0,1,4,9,11$ and $0,3,4,9,11$. For $m=4$ one possible solution is $1,5,6,8$.

In [94]:
m = 5
maxN = 11 # maximum difference allowed

problem = Solver()

letters = 'abcdefghijklmnopqrstuvwxyz'
sequence_vars = Ints(' '.join(letters[:m]))
for var in sequence_vars:
  problem.add(var >= 0)

diff_vars_name = []  # differences between variables
for distance in range(1,m):
  for d in range(m-distance): 
    diff_vars_name.append(str(d)+'-'+str(d+distance))

diff_vars = []
for var_name in diff_vars_name:
  diff_vars.append( Int(var_name) )

for i in range(len(diff_vars)):
  problem.add( diff_vars[i] > 0, diff_vars[i] <= maxN )
  for j in range(i+1,len(diff_vars)):
    problem.add( diff_vars[i] != diff_vars[j] )

# the number sequence must satisfy all difference vars
for var in diff_vars:
  i, j = var.decl().name().split('-')
  i, j = int(i), int(j)
  problem.add( var == sequence_vars[j] - sequence_vars[i] )

problem.check()
m = problem.model()

# show only the sequence solution:
result = []
for d in m.decls():
  if d.name() in letters:
    result.append( int(str(m[d])) )
sorted(result)

[0, 3, 4, 9, 11]



---



## SAT, the Satisfiability Problem

Consider we want to find boolean values for a set of **propositional variables** $c_i$ to evaluate a given  formula to true.

Herein, formulas are presented in [Conjunctive Normal Form](https://en.wikipedia.org/wiki/Conjunctive_normal_form) (CNF).

One CNF formula example,

$$(c_1  \vee c_2 \vee \neg c_6) \wedge \neg c_2 \wedge (c_3 \vee c_5) \wedge c_4 \wedge \neg c_1$$

A **model** (or interpretation) is a mapping of values to every variable.

Given that, the next terms can be defined:

+ A formula is **satisfiable** is there exists at least one model that makes the formula true

+ A formula is **valid** is all possible models make the formula true

+ A formula is **invalid** if at least one model makes it false

+ A formula is **unsatisfiable** (or inconsistent) if there's no model to make it true (otherwise it is consistent)

Given a propositional logical formula, the problem of finding if it is satisfiable is a decidable problem. 

The term SAT refers to the following problem,

<i><center>Given a CNF formula F, is F consistent?</center></i>

SAT is NP-complete. However there are very advanced SAT-solvers that are able to answer quite large instances of SAT, by exploring problem symmetries.


### pycoSAT

In [ ]:
!pip install pycosat

> PicoSAT is a popular SAT solver written by Armin Biere in pure C. This package provides efficient Python bindings to picosat on the C level, i.e. when importing pycosat, the picosat solver becomes part of the Python process itself. For ease of deployment, the picosat source (namely picosat.c and picosat.h) is included in this project. [ref](https://github.com/ContinuumIO/pycosat)

Module `pycosat` assigns each variable to an integer. A negative `-i` value means `not i`. The previous formula is then defined as,

In [97]:
import pycosat as psat

In [100]:
formula = [ [1,2,-6], [-2], [3,5], [4], [-1] ]

Function `solve` outputs a model (if any exists),

In [101]:
psat.solve(formula)

[-1, -2, 3, 4, 5, -6]

`solve` will output `UNSAT` if the formula is unsatisfiable

In [103]:
psat.solve([[1],[-1]]) # x1 /\ ~x1

'UNSAT'

It's possible to output all possible models, `itersolve` returns an iterator for all models,

In [102]:
for sol in psat.itersolve(formula):
  print(sol)

[-1, -2, 3, 4, 5, -6]
[-1, -2, 3, 4, -5, -6]
[-1, -2, -3, 4, 5, -6]


### Example: 8 Queens

In [107]:
# ref: https://github.com/ContinuumIO/pycosat/tree/master/examples
N = 4

def v(i, j):
  return N * i + j + 1

def to_cnf(vs, eq=False):
  if eq:
    yield vs
  for v1 in vs:
    for v2 in vs:
      if v1 < v2:
        yield [-v1, -v2]

def queens_clauses():
  # rows and columns
  for i in range(N):
    yield from to_cnf([v(i, j) for j in range(N)], True)
    yield from to_cnf([v(j, i) for j in range(N)], True)
  # diagonal
  for i in range(N - 1):
    yield from to_cnf([v(j, i + j) for j in range(N - i)])
  for i in range(1, N - 1):
    yield from to_cnf([v(j + i, j) for j in range(N - i)])
  for i in range(N - 1):
    yield from to_cnf([v(j, N - 1 - (i + j)) for j in range(N - i)])
  for i in range(1, N - 1):
    yield from to_cnf([v(j + i, N - 1 - j) for j in range(N - i)])

def solve():
  clauses = queens_clauses()
  # solve the SAT problem
  for sol in psat.itersolve(clauses):
    sol = set(sol)
    for i in range(N):
      print(''.join('Q' if v(i, j) in sol else '.' for j in range(N)))
    print('')

solve()

..Q.
Q...
...Q
.Q..

.Q..
...Q
Q...
..Q.



### Example: Sudoku

In [109]:
# ref: https://github.com/ContinuumIO/pycosat/tree/master/examples
"""
The implementation of this Sudoku solver is based on the paper:
    "A SAT-based Sudoku solver" by Tjark Weber
    https://www.lri.fr/~conchon/mpri/weber.pdf
If you want to understand the code below, in particular the function valid(),
which calculates the 324 clauses corresponding to 9 cells, you are strongly
encouraged to read the paper first.  The paper is very short, but contains
all necessary information.
"""

def v(i, j, d):
    """
    Return the number of the variable of cell i, j and digit d,
    which is an integer in the range of 1 to 729 (including).
    """
    return 81 * (i - 1) + 9 * (j - 1) + d

def sudoku_clauses():
    """
    Create the (11745) Sudoku clauses, and return them as a list.
    Note that these clauses are *independent* of the particular
    Sudoku puzzle at hand.
    """
    res = []
    # for all cells, ensure that the each cell:
    for i in range(1, 10):
        for j in range(1, 10):
            # denotes (at least) one of the 9 digits (1 clause)
            res.append([v(i, j, d) for d in range(1, 10)])
            # does not denote two different digits at once (36 clauses)
            for d in range(1, 10):
                for dp in range(d + 1, 10):
                    res.append([-v(i, j, d), -v(i, j, dp)])

    def valid(cells):
        # Append 324 clauses, corresponding to 9 cells, to the result.
        # The 9 cells are represented by a list tuples.  The new clauses
        # ensure that the cells contain distinct values.
        for i, xi in enumerate(cells):
            for j, xj in enumerate(cells):
                if i < j:
                    for d in range(1, 10):
                        res.append([-v(xi[0], xi[1], d), -v(xj[0], xj[1], d)])

    # ensure rows and columns have distinct values
    for i in range(1, 10):
        valid([(i, j) for j in range(1, 10)])
        valid([(j, i) for j in range(1, 10)])
    # ensure 3x3 sub-grids "regions" have distinct values
    for i in 1, 4, 7:
        for j in 1, 4 ,7:
            valid([(i + k % 3, j + k // 3) for k in range(9)])

    assert len(res) == 81 * (1 + 36) + 27 * 324
    return res


def solve(grid):
    """
    solve a Sudoku grid inplace
    """
    clauses = sudoku_clauses()
    for i in range(1, 10):
        for j in range(1, 10):
            d = grid[i - 1][j - 1]
            # For each digit already known, a clause (with one literal).
            # Note:
            #     We could also remove all variables for the known cells
            #     altogether (which would be more efficient).  However, for
            #     the sake of simplicity, we decided not to do that.
            if d:
                clauses.append([v(i, j, d)])

    # solve the SAT problem
    sol = set(psat.solve(clauses))

    def read_cell(i, j):
        # return the digit of cell i, j according to the solution
        for d in range(1, 10):
            if v(i, j, d) in sol:
                return d

    for i in range(1, 10):
        for j in range(1, 10):
            grid[i - 1][j - 1] = read_cell(i, j)


from pprint import pprint

# hard Sudoku problem, see Fig. 3 in paper by Weber
hard = [[0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 6, 0, 0, 0, 0, 3],
        [0, 7, 4, 0, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 0, 2],
        [0, 8, 0, 0, 4, 0, 0, 1, 0],
        [6, 0, 0, 5, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 7, 8, 0],
        [5, 0, 0, 0, 0, 9, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 4, 0]]
solve(hard)
pprint(hard)
assert [[1, 2, 6, 4, 3, 7, 9, 5, 8],
        [8, 9, 5, 6, 2, 1, 4, 7, 3],
        [3, 7, 4, 9, 8, 5, 1, 2, 6],
        [4, 5, 7, 1, 9, 3, 8, 6, 2],
        [9, 8, 3, 2, 4, 6, 5, 1, 7],
        [6, 1, 2, 5, 7, 8, 3, 9, 4],
        [2, 6, 9, 3, 1, 4, 7, 8, 5],
        [5, 4, 8, 7, 6, 9, 2, 3, 1],
        [7, 3, 1, 8, 5, 2, 6, 4, 9]] == hard

[[1, 2, 6, 4, 3, 7, 9, 5, 8],
 [8, 9, 5, 6, 2, 1, 4, 7, 3],
 [3, 7, 4, 9, 8, 5, 1, 2, 6],
 [4, 5, 7, 1, 9, 3, 8, 6, 2],
 [9, 8, 3, 2, 4, 6, 5, 1, 7],
 [6, 1, 2, 5, 7, 8, 3, 9, 4],
 [2, 6, 9, 3, 1, 4, 7, 8, 5],
 [5, 4, 8, 7, 6, 9, 2, 3, 1],
 [7, 3, 1, 8, 5, 2, 6, 4, 9]]




---



## References

+ Krzysztof Apt - [Principles of Constraint Programming](https://www.amazon.co.uk/dp/0521825830) (2003)

+ Eric Pony - [Z3 in Python, Guide Examples](https://ericpony.github.io/z3py-tutorial/guide-examples.htm)

+ Nikolaj Bjørner et al. - [Programming Z3](https://theory.stanford.edu/~nikolaj/programmingz3.html)

+ Microsoft - [Z3 API](https://z3prover.github.io/api/html/namespacez3py.html)

+ [pycosat: bindings to picosat (a SAT solver)](https://github.com/ContinuumIO/pycosat)